#  DDP

### 1. running on our DGX
- docker run -it --rm --gpus '"device=4,5,6,7"' -p 12344:12344 --name msalehjahromi --shm-size=192G  --user $(id -u):$(id -g) --group-add 1944259512 --cpuset-cpus=49-96 -v /rsrch7/home/ip_rsrch/wulab/:/rsrch7/home/ip_rsrch/wulab -v /rsrch1/ip/msalehjahromi/:/rsrch1/ip/msalehjahromi --name mori_jupyter nnunetv2:msalehjahromi
- cd /rsrch1/ip/msalehjahromi/codes/FineTune/multiGPU/n_GPU_A100
- jupyter notebook --ip 0.0.0.0 --port 12344
- http://1mcprddgx05

In [1]:
!gpustat

2a974e97ed31              Mon Jun  9 14:36:43 2025  535.216.01
[0] NVIDIA A100-SXM4-40GB | 27'C,   0 % |   628 / 40960 MB |
[1] NVIDIA A100-SXM4-40GB | 26'C,   0 % |   626 / 40960 MB |
[2] NVIDIA A100-SXM4-40GB | 25'C,   0 % |   628 / 40960 MB |
[3] NVIDIA A100-SXM4-40GB | 28'C,   0 % |   628 / 40960 MB |


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,3"# "0,1"

import torch
print(torch.cuda.device_count())   # → 2
print(torch.cuda.current_device()) # → 0 (this maps to your original GPU 2)


1
0


### 2. Probably need installation, restart after and run 3

In [3]:
# ## !pip uninstall -q histolab -y
# # Install required packages first
# import os
# import subprocess
# pip_commands = [
#     ["pip", "install","-q", "--extra-index-url", "https://download.pytorch.org/whl/cu117", 
#      "torch==2.0.0", "torchvision==0.15.0", "omegaconf", "torchmetrics==0.10.3", 
#      "fvcore", "iopath", "xformers==0.0.18", "submitit", "numpy<2.0"],
#     ["pip", "install", "-q",  "--extra-index-url", "https://pypi.nvidia.com", "cuml-cu11"],
#     ["pip", "install","-q",  "black==22.6.0", "flake8==5.0.4", "pylint==2.15.0"],
#     ["pip", "install", "-q", "mmsegmentation==0.27.0"],
#     ["pip", "install","-q", "mmcv-full==1.5.0"],
#     ["pip", "install","-q", "nibabel"]
# ]

# for cmd in pip_commands:
#     try:
#         print(cmd)
#         subprocess.run(cmd, check=True)
#     except subprocess.CalledProcessError as e:
#         print(f"Failed to install packages with command: {cmd}")
#         print(f"Error: {e}")

### 3. running, notice important hyperparams
-  which model are use so vmin -1000., vmax +150, eps 0.00005
- `num-workers`
- `epochs`: 100
- `accum-steps`: 2000 ?
- `max-chunks`: 66 
- `lr`:  lr*0.1 on base ?
- `warmup-steps`: 5k
- `print-every` : 5000
- `val-every` : 40000

In [4]:
## Install required packages first
n_GPU = "1"
import os
import subprocess
# Set required environment variables
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "29500"
os.environ["RANK"] = "0"
os.environ["WORLD_SIZE"] = n_GPU
os.environ["LOCAL_RANK"] = "0"
os.environ["LOCAL_WORLD_SIZE"] = n_GPU

# Build the command with --install-packages flag removed
command = [
    "python3",
    "/rsrch1/ip/msalehjahromi/codes/FineTune/multiGPU/n_GPU_A100_v2/2_run_fineTune_ddp_launcher.py", ##changed
    "--num-gpus", n_GPU,
    "--csv", "/rsrch1/ip/msalehjahromi/codes/FineTune/nlst_event_train_val_.csv",
    "--accum-steps", "200", ###############
    "--num-workers", "10",
    "--epochs", "200", ###############
    "--lr", "0.000008", ###############
    "--warmup-steps", "100", ##########
    "--weight-decay", "0.01",
    "--optimizer", "adamw",
    "--num-attn-heads", "3",
    "--num-layers", "2",
    "--dropout", "0.35",
    "--unfreeze-strategy", "all",  
    "--output", "/rsrch1/ip/msalehjahromi/codes/FineTune/multiGPU/n_GPU_A100_v2/output_ddp_p16_23_1150_ac200_1GPU",
    "--print-every", "10000", ################ 2000
    "--val-every", "40000", ################ 50k
    "--metrics-dir", "/rsrch1/ip/msalehjahromi/codes/FineTune/multiGPU/metrics_multi_gpu"
]

# Run the command
try:
    subprocess.run(command, check=True)
except subprocess.CalledProcessError as e:
    print(f"Error occurred: {e}")
    print(f"Command output: {e.output if hasattr(e, 'output') else 'No output available'}")

/rsrch1/ip/msalehjahromi/codes/dinov2-torchrun-dataloader6/dinov2/layers/swiglu_ffn.py:43: UserWarning: xFormers is available (SwiGLU)
  warnings.warn("xFormers is available (SwiGLU)")
/rsrch1/ip/msalehjahromi/codes/dinov2-torchrun-dataloader6/dinov2/layers/attention.py:27: UserWarning: xFormers is available (Attention)
  warnings.warn("xFormers is available (Attention)")
/rsrch1/ip/msalehjahromi/codes/dinov2-torchrun-dataloader6/dinov2/layers/block.py:33: UserWarning: xFormers is available (Block)
  warnings.warn("xFormers is available (Block)")
2025-06-09 14:36:54,001 - INFO - Starting training on 1 GPUs with full model copies (DDP)


Files in the directory: ['n_GPU_A100_v2', '._.gitignore', '._.DS_Store', '._See_attention.ipynb', '1_GPU_H100', 'See_attention.ipynb', 'n_GPU_A100', 'metrics_single_gpu', 'n_H100onH100', '.DS_Store', '1_GPU_A100', 'ReadMe.md', '.gitignore', 'metrics_multi_gpu', '.git']
Single-GPU run – DDP will run on one device
Train dataset created on rank 

==== Dataset Statistics ====
Total samples: 43852
Training samples: 38382
Validation samples: 5470

1-year-cancer:
  Train positive: 1428 (3.72%)
  Val positive: 188 (3.44%)

2-year-cancer:
  Train positive: 1891 (4.93%)
  Val positive: 256 (4.68%)

3-year-cancer:
  Train positive: 2282 (5.95%)
  Val positive: 315 (5.76%)

4-year-cancer:
  Train positive: 2586 (6.74%)
  Val positive: 363 (6.64%)

5-year-cancer:
  Train positive: 2838 (7.39%)
  Val positive: 403 (7.37%)

6-year-cancer:
  Train positive: 2988 (7.78%)
  Val positive: 429 (7.84%)

Total validation samples: 5470
Created distributed validation loader, each rank processes ~5136 samples


2025-06-09 14:36:55,588 - INFO - using MLP layer as FFN


Model loaded on rank 
[resume] no model_epoch_*.pt — fresh run
Using unweighted BCEWithLogitsLoss
Will save metrics to: /rsrch1/ip/msalehjahromi/codes/FineTune/multiGPU/metrics_multi_gpu/training_metrics_nGPU_DDP_20250609_143710.jsonl
Will save checkpoints to /rsrch1/ip/msalehjahromi/codes/FineTune/multiGPU/n_GPU_A100_v2/output_ddp_p16_23_1150_ac200_1GPU/checkpoints

Training Configuration:
Max chunks per sample: 72
Learning rate: 8e-06
Gradient accumulation steps: 200
Number of tasks: 6
Validation frequency: 40000 steps
Number of epochs: 200
Warmup steps: 100
World size: 1
Device: cuda

Epoch 1: All parameters already unfrozen (strategy: all)


/rsrch1/ip/msalehjahromi/codes/FineTune/multiGPU/n_GPU_A100_v2/2_run_fineTune_ddp_full.py:133: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  if hasattr(m, n) and getattr(m, n).storage().size() == 0:
2025-06-09 14:37:10,386 - INFO - ------ Training Configuration ------
2025-06-09 14:37:10,386 - INFO - Max chunks per sample         : 72
2025-06-09 14:37:10,386 - INFO - Learning rate (aggregator)    : 8e-06
2025-06-09 14:37:10,386 - INFO - Learning rate (base)          : 8e-08
2025-06-09 14:37:10,387 - INFO - Gradient accumulation steps   : 200
2025-06-09 14:37:10,387 - INFO - Warm-up steps                 : 100
2025-06-09 14:37:10,390 - INFO - Aggregator layers / heads     : 2 / 3
2025-06-09 14:37:10,390 - INFO - Aggregator dropout            : 0.35
2

KeyboardInterrupt: 